In [ ]:
# install packages
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install evalml

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd

# evalml
import evalml
from evalml import AutoMLSearch
from evalml.automl import make_data_splitter

import catboost
import lightgbm
import xgboost

from sklearn.preprocessing import LabelEncoder
from joblib import load, dump

In [3]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/colab/TPS Jun"
path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - Jun 2021'
os.chdir(path)

In [9]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [10]:
# target labelling
f = LabelEncoder()
f.fit(df_train['target'])
y = f.transform(df_train['target'])
y = pd.DataFrame(y).astype('int64')

col = [i for i in df_train.columns if i not in ['id', 'target']]
x = df_train[col]
x = x.astype('int64')

In [8]:
# # 可處理的問題
# evalml.problem_types.ProblemTypes.all_problem_types

# # 可使用的模型
# evalml.pipelines.components.utils.allowed_model_families("multiclass")

# # 可使用的目標函數
# evalml.objectives.get_all_objective_names()

In [ ]:
# construct model
# AutoML : evalml
PROJECT_NAME = 'evalml-20210611'
SEED = 202106011

# X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(x, y.squeeze(), problem_type='multiclass')
cv = make_data_splitter(x, y, problem_type='multiclass', n_splits=5, shuffle=True, random_seed=SEED)

automl = AutoMLSearch(X_train=x, 
                      y_train=y.squeeze(), 
                      problem_type="multiclass",   # or 'binary', 'regression', 'time series regression', 'time series binary', 'time series multiclass'
                      objective="log loss multiclass", 
                      allowed_model_families=['DECISION_TREE', 'CATBOOST', 'RANDOM_FOREST' , 'XGBOOST', 'LINEAR_MODEL', 'LIGHTGBM', 'EXTRA_TREES'],
                      data_splitter=cv,
                      random_seed=SEED,
                      n_jobs=-1,
                      max_batches=5)

In [ ]:
# training model
automl.search()

In [ ]:
# save
automl.save(f'{PROJECT_NAME}.pkl')

In [ ]:
# load well-trained model
automl = AutoMLSearch.load(f'{PROJECT_NAME}.pkl')

In [ ]:
# # 全部pipeline表現的排序
# automl.full_rankings

# # 前九名pipeline表現的排序
# automl.rankings

# # 查看第n個pipeline的內容
# automl.get_pipeline(0)

# # 詳細內容
# automl.results

# # 表現最佳的pipeline
# best_pipeline = automl.best_pipeline

# # 以graphviz呈現pipeline
# best_pipeline.graph()

# # testing set
# scores = best_pipeline.score(X_test, y_test, 
#                              objectives=evalml.objectives.get_core_objectives('multiclass'))

# scores['Log Loss Multiclass']

In [ ]:
# fit best pipeline
best_pipeline.fit(x, y.squeeze())

In [ ]:
# save best pipeline
best_pipeline.save(f'{PROJECT_NAME}_best.pkl')

In [ ]:
# load best pipeline
best_pipeline = automl.load(f'{PROJECT_NAME}_best.pkl')

In [ ]:
# predict probability
x_test = df_test.drop(['id'], axis=1)
x_test = x_test.astype('int64')
result = best_pipeline.predict_proba(x_test).to_dataframe()

In [ ]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result
sub.to_csv(f'{RESULTS_PATH}.csv', index=False)